## file name: run_model
### @author: Sacha Gavino
### last update: jan 2025
### language: PYTHON 3.12
__________________________________________________________________________________________
#### short description:  create thermal files
__________________________________________________________________________________________

In [3]:
import chemdiskpy.modeling as modeling
import chemdiskpy.dust as dust
import chemdiskpy.plotting.plot as plot

import numpy as np

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [4]:
from chemdiskpy.constants.constants import autocm, kb, mu, amu, Ggram, M_sun

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [6]:
m = modeling.Structure()  

In [7]:
#-----DISK PARAMETERS MODEL THIN
nr, ntheta, nphi = 301, 181, 2
rin, rout= 2, 350
ref_radius=100
Rdrift, lsx, llx = 150, 1, 1e9
lx = None
dtogas = 1e-2
star_mass = 0.60
star_temp = 4000
star_lum = 0.75
#dust_mass = 5e-5
sigma_gas_ref= 0.63 #0.2518 for M=7.45e-5 with rout=500
h0 = 8.6
q_c = 12
q_exp = 0.4
alpha = 1e-2
Tmidplan_ref=10

In [8]:
sig_h2 = 10**(23.2)
sig_g = sig_h2 * amu * mu

In [9]:
sig_g

0.631626439229934

In [7]:
#-----CREATE WAVELENGTH TABLE----
m.grid.set_wavelength_grid(9.12e-2, 1e4, 200, log=True) #microns
m.grid.set_mcmonowavelength_grid(9.12e-2, 2e-1, 10, log=True) #microns

In [8]:
#-----CREATE RADMC GRID-----
m.set_spherical_grid(rin, rout, nr, ntheta, nphi, log=True)

In [9]:
# #-----CREATE STARS-----
m.add_star(mass=star_mass, luminosity=star_lum, temperature=star_temp, x=0., y=0., z=0.)

In [10]:
# # #-----CREATE ISRF-----
m.add_isrf(cut=2.e-1, d78=True, vdb82=True)

In [13]:
# # -----CREATE DUST DENSITIES-----
d = dust.MRNDistrib(rsingle=0.1, amin=5.000e-03, amax=1.000e+03, nb_sizes=2, rho_m=2., llx=llx, lsx=lsx, lx=lx) #microns
sizes = d.sizes()
m.grid.add_dust(d)
frac = d.massfraction()
sizes
d.a

AttributeError: 'MRNDistrib' object has no attribute 'a'

In [12]:
m.add_disk(dust=d, rin=rin, rout=rout, Rdrift=Rdrift, ref_radius=ref_radius, sigma_gas_ref=sigma_gas_ref, h0=h0, star_mass=star_mass, q_exp= q_exp, alpha=alpha, dtogas=dtogas, q_c=q_c, settling=True, coordsystem='spherical')


Radial drift is added:

Drifted mass (solar mass):  6.391334150910224e-21

mass fraction of large grains inside and outside:  0.9548480246411201 and 0.9548480246411201
[[0.04515198 0.95484802]
 [0.04515198 0.95484802]]


In [13]:
# -----RUN THERMAL-----
m.thermal(run=False, nphot = 8e6, \
              write_dens=True, \
              write_grid=False, \
              write_opac=True, \
              write_control=True, \
              write_star=False, \
              write_wave=False, \
              write_mcmono=False, \
              write_ext=False, \
              nphot_scat = 8e6, \
              nphot_spec = 1e5, \
              itempdecoup = 1, \
              istar_sphere = 1, \
              modified_random_walk = 1, \
              rto_style = 1, \
              scattering_mode_max = 2, \
              tgas_eq_tdust = 0)


WRITING RADMC3D INPUT FILES:
----------------------------

number of structures (disk, envelope...):  1 

number of species in structure 1:  2
total number of grain species:  2
Some RADMC3D input files will not be created. Will continue... but errors can be raised if one or more required input files are missing.

Dust thermal simulation was not run because the user set 'run=False'. This is fine. Will continue.
Unless the file dust_temperature.dat does not exist, in that case, set 'run=True' and come back.

No dust temperature file was found. If coupling_temp is True the chemistry model cannot be created.




In [25]:
m.localfield(run=False, nphot_mono = 8000000)


WRITING RADMC3D INPUT FILES:
----------------------------



FileNotFoundError: [Errno 2] No such file or directory: 'thermal/mean_intensity.out'

In [13]:
rin = np.arange(19, 150, 2)
rout = np.arange(151, 356, 5)
rchem = np.concatenate((rin, rout), axis=0)
msize = rchem[-1] #model boundary

In [14]:
m.grid.set_chem_grid(r=rchem, z0=0.1, msize=msize, nbcells=90)

In [15]:

m.write_nautilus(stop_time=3e6, nb_outputs = 62, is_h2_formation_rate=1, temp_gas='dust', tunneling=1, static=True, param=True, element=True, abundances='abundances_coldcore.in', \
                      network=True, multi_grain=True, tempdecoup=True,\
                      coupling_dens=True, coupling_temp=True, coupling_av=True)

WRITING NAUTILUS INPUT FILES:
-----------------------------

Multi-grain mode: Yes

Dust temperature structure: multiple dust temperatures (dust parameters stored in 1D_grain_sizes.in).



In [1]:
0.01/1.01

0.009900990099009901

In [3]:
0.009900990099009901/(1-0.009900990099009901)

0.01

In [1]:
1.7e-4/2.4e-4

0.7083333333333334

In [29]:
idust=0
amin = 1e-2
amax = 10
ndust = 10



def single_interval(idust, ndust, amin, amax):
    return 10**(np.log10(amax/amin)*((idust-1)/ndust)+np.log10(amin))

# def average(idust):
#     return np.sqrt(intervals[idust-1] * intervals[idust])
# a = [average(idust) for idust in range(1, ndust+1)]

intervals = np.array([single_interval(idust, ndust, amin, amax) for idust in range(1, ndust+2)])
av = np.sqrt(intervals[1:] * intervals[:-1])





In [32]:
intervals[1:], intervals[:-1], intervals

(array([ 0.01995262,  0.03981072,  0.07943282,  0.15848932,  0.31622777,
         0.63095734,  1.25892541,  2.51188643,  5.01187234, 10.        ]),
 array([0.01      , 0.01995262, 0.03981072, 0.07943282, 0.15848932,
        0.31622777, 0.63095734, 1.25892541, 2.51188643, 5.01187234]),
 array([ 0.01      ,  0.01995262,  0.03981072,  0.07943282,  0.15848932,
         0.31622777,  0.63095734,  1.25892541,  2.51188643,  5.01187234,
        10.        ]))

In [1]:
a = np.logspace(np.log10(amin), np.log10(amax), ndust+1)

NameError: name 'np' is not defined